In [ ]:
!pip install requests beautifulsoup4 openpyxl

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from openpyxl import Workbook
from bs4 import BeautifulSoup
import re
from google.colab import files

# Configuración
MAX_WORKERS = 20  # Ajusta este número según tus necesidades y las limitaciones del sitio web
TIMEOUT = 10  # Tiempo de espera para cada solicitud en segundos

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def obtener_datos_producto(codigo_padre):
    url = f'https://www.marathon.cl/{codigo_padre}.html'
    try:
        response = requests.get(url, headers=headers, timeout=TIMEOUT)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        precio_actual = soup.select_one('span.value[content]')
        precio_actual = precio_actual['content'] if precio_actual else 'N/A'
        
        precio_anterior = soup.select_one('span.value[content]:nth-of-type(2)')
        precio_anterior = precio_anterior['content'] if precio_anterior else 'N/A'
        
        descuento = soup.select_one('div.pd-item-promo')
        descuento = descuento.text.strip() if descuento else 'N/A'
        
        imagenes = len(soup.select('img.galley_img'))
        
        return codigo_padre, precio_actual, precio_anterior, descuento, imagenes
    except Exception as e:
        print(f"Error procesando {codigo_padre}: {str(e)}")
        return codigo_padre, 'N/A', 'N/A', 'N/A', 'N/A'

def procesar_codigos(codigos):
    resultados = []
    total_codigos = len(codigos)
    start_time = datetime.now()
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(obtener_datos_producto, codigo): codigo for codigo in codigos}
        for i, future in enumerate(as_completed(futures), 1):
            resultados.append(future.result())
            elapsed_time = datetime.now() - start_time
            tiempo_transcurrido = str(elapsed_time).split('.')[0]
            print(f"Procesando código {i}/{total_codigos} - Tiempo transcurrido: {tiempo_transcurrido}")
    
    return resultados

def guardar_resultados(resultados):
    wb = Workbook()
    ws = wb.active
    ws.title = "Datos Productos"
    ws['A1'] = "CODIGO"
    ws['B1'] = "PRECIO ACTUAL"
    ws['C1'] = "PRECIO ANTERIOR"
    ws['D1'] = "DESCUENTO"
    ws['E1'] = "Cant. Img"
    
    for i, (codigo, precio_actual, precio_anterior, descuento, cantidad_imagenes) in enumerate(resultados, start=2):
        ws[f'A{i}'] = codigo
        ws[f'B{i}'] = precio_actual
        ws[f'C{i}'] = precio_anterior
        ws[f'D{i}'] = descuento
        ws[f'E{i}'] = cantidad_imagenes

    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo = f"Datos Productos Web {fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    print(f"Archivo '{nombre_archivo}' guardado con los precios, descuentos y cantidad de imágenes de los productos.")
    files.download(nombre_archivo)

# Celda 1: Bloque adicional para ingresar todos los códigos padres
codigos_input = """
10803421016
10962435018
10962438016
10962595019
10875176015
10911276014
10966099022
10962821013
10911276017
10962822013
10962821014
10866961002
10962821019
10962438013
10962692014
10875176013
10962435019
10962822016
10962438018
10966099018
10962594013
10962736022
10962389018
10930745010
10962389016
"""  # Reemplazar con los códigos pegados desde un Excel, separados por espacios
codigos = codigos_input.strip().split()

# Ejecutar el script
resultados = procesar_codigos(codigos)
guardar_resultados(resultados)